## 파이썬으로 말뭉치 전처리 하기 (말뭉치 corpus 사람이 쓰는 단어들의 집합)

In [4]:
text = 'You say goodbye and I say hello.' # 말뭉치 하나

In [5]:
text = text.lower() 

In [6]:
text = text.replace('.', ' .') # 구두점을 따로 분리, 구두점도 단어로 인식하기 때문 

In [7]:
text

'you say goodbye and i say hello .'

In [8]:
words = text.split(' ') # 각 단어로 분리

In [9]:
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

###  딕셔너리를 이용하여 단어 ID와 단어를 짝지어 주는 대응표 작성

In [10]:
word_to_id = {} 
id_to_word = {} 

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id) # 워드의 갯수가 곧 아이디가 된다 최초시 word_to_id = {0:'you'} "say" 두번이지만 duplicate는 고려하지 않음
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [11]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [12]:
word_to_id # encode

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [13]:
id_to_word[1] # decode

'say'

In [14]:
word_to_id['hello']

5

In [15]:
import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

###  말뭉치를 이용하기 위한 전처리 함수 구현

In [16]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word # 문장을 넣으면 숫자로 바뀌어서 출력

In [17]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [25]:
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [26]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [27]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

### 동시발생 행렬

In [28]:
import sys
sys.path.append('..')
import numpy as np
from common.util import preprocess
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [29]:
print(corpus)

[0 1 2 3 4 1 5 6]


In [30]:
print(id_to_word)

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [31]:
C = np.array([
    [0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0],
], dtype=np.int32)

In [32]:
print(C[0])

[0 1 0 0 0 0 0 0]


In [33]:
print(C[4])

[0 1 0 1 0 0 0 0]


In [34]:
print(C[word_to_id['goodbye']])

[0 1 0 1 0 0 0 0]


In [29]:
# 동시발생행렬 자동생성
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

### 벡터 간 유사도

In [35]:
def cos_similarity(x, y):
    nx = x / np.sqrt(np.sum(x ** 2))
    ny = y / np.sqrt(np.sum(y ** 2))
    return np.dot(nx, ny)

In [37]:
def cos_similarity(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps) # 위와 같으나 분모가 0이 되면 안되기 때문에 미세한 eps를 더함
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)

In [44]:
import sys
sys.path.append('..')
from common.util import preprocess, create_co_matrix, cos_similarity


text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
#전처리 끝

vocab_size = len(word_to_id) 
print(vocab_size) # 7
C = create_co_matrix(corpus, vocab_size) #Co-matrix 동시발생행렬
print(C)
c0 = C[word_to_id['you']]  # "you"의 단어 벡터
c1 = C[word_to_id['i']]    # "i"의 단어 벡터

print(c0)
print(c1)
print(cos_similarity(c0, c1)) # "you" 하고 "i" 는 약 70% 유사한 단어이다

[0 1 2 3 4 1 5 6]
7
[[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
[0 1 0 0 0 0 0]
[0 1 0 1 0 0 0]
0.7071067691154799


### 유사 단어의 랭킹 표시

In [45]:
import numpy as np
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return

    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # 코사인 유사도 계산
    vocab_size = len(id_to_word)

    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # 코사인 유사도를 기준으로 내림차순으로 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

In [46]:
x = np.array([100, -20, 2])

In [47]:
x.argsort()

array([1, 2, 0], dtype=int64)

In [48]:
(-x).argsort()

array([0, 2, 1], dtype=int64)

In [49]:
import sys
sys.path.append('..')
from common.util import preprocess, create_co_matrix, most_similar


text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0
